In [1]:
import requests
from bs4 import BeautifulSoup
import csv

resp = requests.get('https://www.trustpilot.com/review/canadiantire.ca')
html_code = resp.text

# save it so that it can be viewed in an editor
with open('resp.html', 'w') as f:
    f.write(resp.text)


In [2]:
soup = BeautifulSoup(html_code, 'html.parser')
print(soup.title)

<title>Canadian Tire Reviews | Read Customer Service Reviews of canadiantire.ca</title>


In [ ]:
# Extract the total number of reviews
print(soup.find_all('h2'))

In [4]:
num_reviews=soup.select('h2')[0].text
import re
p = re.compile(R"[0-9]{1,5}")
m = p.search(num_reviews)
total = m.group()
if m:
    print('Total number of review:', total)
else:
    print("No reviews yet")

Total number of review: 766


In [5]:
# Extract the total number of reviews
total_reviews=soup.find('span', class_ = 'headline__review-count')
total_reviews.text

'766'

In [6]:
def get_next_page(soup):
    container = soup.find('a', {'rel': 'next'})
    if not container:
      return
    return 'https://www.trustpilot.com' + container['href']

In [7]:
url = 'http://www.trustpilot.com/review/canadiantire.ca'
link=[]
while True:
    page = requests.get(url)
    page_soup = BeautifulSoup(page.text, 'html.parser')
    url = get_next_page(page_soup)
    link.append(url)
    if not url:
        break


In [8]:
URL = link[:-1]
URL.insert(0,'http://www.trustpilot.com/review/canadiantire.ca')

In [9]:
URL

['http://www.trustpilot.com/review/canadiantire.ca',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYyMDI0NjUzMTAwMHw2MDkzMDAwM2Y5ZjQ4NzBhNzg2ZmE2N2E',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYxOTQ1Mjc2NzAwMHw2MDg2ZTM1ZmY5ZjQ4NzAxZmM3ZTE1MzA',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYxODI4MzAyNjAwMHw2MDc1MGExMmY4NWQ3NTA4NzA1OGJiNzU',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYxNjcxNTA5NDAwMHw2MDVkMWQ1NmY4NWQ3NTA4NzAzNTc0MjI',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYxNDQ2NzU5NDAwMHw2MDNhZDIwYWY4NWQ3NTBiOThhMmNlZWU',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYxMjU1MTMzMDAwMHw2MDFkOTRhMjY3OWQ5NzA4ZDgyNmZlM2M',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYxMDMzODQ0MDAwMHw1ZmZiZDA4ODc1NWRjMTA3ZTBkZDI2NDQ',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYwOTIwNTgxNDAwMHw1ZmVhODgzNjc1NWRjMTA3ZTBkMTM1Y2U',
 'https://www.trustpilot.com/review/canadiantire.ca?b=MTYwNjg0MTQzNDAwMHw1ZmM2NzQ1Y

In [10]:
company = ['Canadian Tire' for i in range(int(total))]
review = []
date = []
rating= []

for x in URL: 
  page = requests.get(x)
  page_soup = BeautifulSoup(page.text, 'html.parser')
  for i in page_soup.find_all('div',{'class':'review-content__body'}):
    per_review = i.find('p',{'class':'review-content__text'})
    review.append(str(per_review))
  for i in page_soup.find_all('div',{'class':'review-content-header__dates'}):
    per_date = i.find('script', { 'data-initial-state':'review-dates'})
    per_date = per_date.text
    p = re.compile(R'([0-9]{4}\-[0-9]{2}\-[0-9]{2}T[0-9]{2}\:[0-9]{2}\:[0-9]{2}\+[0-9]{2}\:[0-9]{2})')
    m = p.search(per_date)
    per_date = m.group()
    date.append(str(per_date))
  for i in page_soup.find_all('div',{'class':'star-rating star-rating--medium'}):
    per_rating = str(i.find('img'))
    p = re.compile(R'([0-9]{1})')
    m = p.search(per_rating)
    per_rating = m.group()
    rating.append(str(per_rating))

In [11]:
new_reviews = []
for i in review:
  new_i = str(i).replace('<br/>','')
  new_i = new_i[35:-17]
  new_reviews.append(new_i)
new_reviews

['              Horrible. If you could rate a ZERO. Owen Sound would get a zero.  Called to placr a curbside order, the staff informed me that I DID NOT have to "CALL IT IN" Weird but ok. Just let the staff know and they will get it for you! Well, I waited in line and was then told oh you have to place that on phone order.....totally unprofessional or very lazy staff... will take my business elsewhere. Customer service sucks.!',
 '              This could be called: "HIGHWAY ROBBERY"In 2019 My daughter borrowed my 2004 Mazda Tribute to travel from Toronto to Brockville.  Not wanting to use my vehicle, she used her own.  Three months elapsed without the Mazda being used.  On my request she went to start the car without success.  She boosted the car, it started and went around for few Km.  Parked the car.  Next day the car would not start again.  On March 7,2019 I went to pick up the car.  No start.  Boosted it went to Canadian tire Brockville, after making appointment, the car arrived o

In [12]:
# open a new file as follows:
f = open("CT_reviews.csv", 'w')
# close it with f.close()
# start a csv writer object:
writer = csv.writer(f, delimiter=',')
# write a row, this writes the header
writer.writerow(["CompanyName", "DatePublished","RatingValue","ReviewBody"])

for i in range(len(review)):
  writer.writerow([company[i], date[i],rating[i],new_reviews[i]])


f.close()

# you can extract the text node with link.contents
# and you can get the actual link href with link.get('href')